이번 실습을 통해, 텐서플로우로 구현된 RNN을 직접 탐구해보도록 하겠습니다.

In [1]:
import numpy as np
from IPython.display import Image

기본적으로 RNN은 아래의 다이어그램과 같이 구현됩니다.  
보시다시피, RNN은 입력값(x), 출력값(output), 상태값(state), 가중치(W), 편향값(b), 그리고 활성화함수(tanh)로 구성되어 있습니다.

In [4]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_rnn.png", width=450, height=200)

# RNN 텐서플로우 코드 구현
위에서 다뤄보았던 예제를 직접 텐서플로우로 구현하여 입력값, 출력값, 상태값, 편향값을 확인해보도록 하겠습니다.

In [6]:
import tensorflow as tf

In [7]:
inputs = np.array([
    [ [1,2] ]
])

In [8]:
tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype=tf.float32)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(
    cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)
variables_names =[v.name for v in tf.trainable_variables()]

print(outputs)
print(state)
print("weights")
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print("output values")
    print(output_run)
    print("\nstate value")
    print(state_run)
    print("weights")
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k, v)

Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
output values
[[[-0.9314169   0.75578666 -0.6819246 ]]]

state value
[[-0.9314169   0.75578666 -0.6819246 ]]
weights
rnn/basic_rnn_cell/kernel:0 [[-0.62831575  0.38538355  0.79733914]
 [-0.5203329   0.30046564 -0.8150209 ]
 [ 0.39399797  0.16670114  0.4062907 ]
 [-0.6391754   0.8460203   0.5266966 ]
 [ 0.41124135  0.66347724 -0.0210759 ]]
rnn/basic_rnn_cell/bias:0 [0. 0. 0.]


# RNN 텐서플로우 코드 출력 이해하기
위 코드의 출력값을 통해, 텐서플로우 RNN이 우리가 이론으로 다룬 RNN과 같은 것을 확인할 수 있습니다..  
1. RNN 셀이 한개일 경우 그 출력값과 상태값이 동일합니다.  
   output values [[[-0.9314169   0.75578666 -0.6819246 ]]]  
   state value   [[-0.9314169   0.75578666 -0.6819246 ]]  
   
   
2. 입력값이 1x2행렬이고, RNN의 상태값이 1x3의 행렬일 경우, **W**는 총 5개의 행을 가지게 됩니다.  
   [[-0.62831575  0.38538355  0.79733914]  
    [-0.5203329   0.30046564 -0.8150209 ]  
    [ 0.39399797  0.16670114  0.4062907 ]  
    [-0.6391754   0.8460203   0.5266966 ]  
    [ 0.41124135  0.66347724 -0.0210759 ]]  
  
  
3. 입력값이 1x2행렬이고, RNN의 상태값이 1x3의 행렬일 경우, 편향값은 총 3개가 필요합니다.  
   rnn/basic_rnn_cell/bias:0 [0. 0. 0.]

# 텐서플로우로 단어 품사 구분하기 예제 구현하기
이론에서 다뤘던 "I work at google", "I google at work"의 단어 품사를 구분하는 RNN 코드를 텐서플로우로 구현해보도록 하겠습니다.  
각 단어는 one hot encoding으로 표현합니다.

In [9]:
# I      [1,0,0,0]
# work   [0,1,0,0]
# at     [0,0,1,0]
# google [0,0,0,1]
#
# I work at google =  [ [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1] ]
# I google at work =  [ [1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0] ]

inputs = np.array([
    [ [1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1] ],
    [ [1,0,0,0], [0,0,0,1], [0,0,1,0], [0,1,0,0] ]
])

아래의 코드를 실행하여, 결과값을 확인합니다.

In [10]:
tf.reset_default_graph()
tf.set_random_seed(777)
tf_inputs = tf.constant(inputs, dtype=tf.float32)
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=3)
outputs, state = tf.nn.dynamic_rnn(
    cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs)
variables_names =[v.name for v in tf.trainable_variables()]

print(outputs)
print(state)
print("weights")
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
        
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print("output values")
    print(output_run)
    print("\nstate value")
    print(state_run)
    print("weights")
    values = sess.run(variables_names)
    for k,v in zip(variables_names, values):
        print(k, v)

Tensor("rnn/transpose_1:0", shape=(2, 4, 3), dtype=float32)
Tensor("rnn/while/Exit_3:0", shape=(2, 3), dtype=float32)
weights
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(7, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>
output values
[[[-0.50944704  0.33166462  0.6126557 ]
  [-0.20793891  0.24406303 -0.75278705]
  [-0.06346128 -0.52844936  0.68356085]
  [-0.36491966  0.8857268  -0.02324398]]

 [[-0.50944704  0.33166462  0.6126557 ]
  [-0.30707452  0.62735885  0.21719742]
  [ 0.5043804  -0.14038289  0.3744523 ]
  [-0.11641283  0.70696247 -0.7512605 ]]]

state value
[[-0.36491966  0.8857268  -0.02324398]
 [-0.11641283  0.70696247 -0.7512605 ]]
weights
rnn/basic_rnn_cell/kernel:0 [[-0.56198275  0.34469748  0.7131618 ]
 [-0.4653999   0.2687447  -0.7289769 ]
 [ 0.35240245  0.14910203  0.36339748]
 [-0.57169586  0.7567036   0.47109187]
 [ 0.3678255   0.5934322  -0.01885086]
 [ 0.31208777 -0.40880746  0.22867584]
 [ 0.5521256   0.682691   -0

# 텐서플로우 코드 출력 이해하기
1. 두 문장의 첫단어 출력값이 동일합니다. 이는 첫단어에는 이전 상태값이 존재하지 않기 때문입니다.  
   [-0.50944704  0.33166462  0.6126557 ]  
   
2. 두번째 단어부터의 출력값은 두 문장이 다르게 나타납니다. 이는 이전 상태값이 현재 출력값에 영향을 주기 때문입니다.
3. 상태값은 항상 마지막 단어의 출력값과 동일합니다.  
   첫번째 문장: [-0.36491966  0.8857268  -0.02324398]  
   두번째 문장: [-0.11641283  0.70696247 -0.7512605 ]